In [1]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
from datetime import datetime

In [2]:
import config
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

https://stackoverflow.com/questions/40023931/how-to-get-real-estate-data-with-idealista-api

In [3]:
def get_oauth_token(): 
    url = "https://api.idealista.com/oauth/token"    
    apikey= API_KEY #sent by idealist
    secret= API_SECRET #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]
    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [4]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [5]:
# values: es, it, pt
country = 'es' 
# search language for summary. values: es, it, pt, en, ca
locale = 'es' 
language = 'es' 
# items per page: 50 as maximum allowed
max_items = '50'
# values: sale, rent
operation = 'sale'
# values: homes, offices, premises, garages, bedrooms
property_type = 'homes'
# allowed values by property type:
# garages: distance, price, street, photos, publicationDate, modificationDate (rent only), weigh, pricedown
# premises: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2 (rent only), weigh, pricedown
# offices: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2, weigh, pricedown
# homes: distance, price, street, photos, publicationDate, modificationDate, size, floor, rooms, ratioeurm2 (sólo alquiler), weigh, pricedown
# rooms: distance, price, street, photos, publicationDate, modificationDate, floor
order = 'priceDown' 
# geographic coordinates (WGS84) (latitude,longitude)
center = '43.3773,-8.39852' 
# distance to center, in metres (ratio)
distance = '5000'
# values: asc, desc
sort = 'desc'
# 
maxPrice = '300000'
# numPage = 1

In [6]:
df = pd.DataFrame()
for i in range(1,15):
    numPage = str(i)
    url = ('https://api.idealista.com/3.5/'+country+
            '/search?operation='+operation+
            '&locale='+locale+
            '&maxItems='+max_items+
            '&order='+order+
            '&center='+center+
            '&distance='+distance+
            '&maxPrice='+maxPrice+
            '&propertyType='+property_type+
            '&sort='+sort+ 
            '&numPage='+numPage+
            '&language='+language)
    result = search_api(get_oauth_token(), url)
    df_temp = pd.json_normalize(result, record_path = ["elementList"])
    df = pd.concat([df, df_temp])
now = datetime.now()
date_time = now.strftime("%m%d%Y_%H:%M")
df.to_csv('idealista_search' + str(date_time) + '.csv')

In [7]:
print(len(df))
df

700


,propertyCode,thumbnail,numPhotos,floor,price,propertyType,operation,size,exterior,rooms,...,suggestedTexts.subtitle,suggestedTexts.title,externalReference,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,neighborhood,detailedType.subTypology,labels,parkingSpace.parkingSpacePrice,newDevelopmentFinished
0,95334761,https://img3.idealista.com/blur/WEB_LISTING/0/...,18,2,148000.0,flat,sale,93.0,True,4,...,"Los Castros - Castrillón, A Coruña",Piso en Oza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,95313523,https://img3.idealista.com/blur/WEB_LISTING/0/...,9,5,145000.0,flat,sale,56.0,False,1,...,"Los Castros - Castrillón, A Coruña",Piso,10006,True,True,NaN,NaN,NaN,NaN,NaN
2,100677141,https://img3.idealista.com/blur/WEB_LISTING/0/...,45,bj,210000.0,duplex,sale,105.0,False,2,...,"Falperra-Santa Lucía, A Coruña",Dúplex en Doctor Fleming,JM-335,NaN,NaN,Falperra-Santa Lucía,duplex,NaN,NaN,NaN
3,100006630,https://img3.idealista.com/blur/WEB_LISTING/0/...,24,bj,210000.0,duplex,sale,105.0,False,2,...,"Falperra-Santa Lucía, A Coruña",Dúplex en Doctor Fleming,NaN,NaN,NaN,Falperra-Santa Lucía,duplex,NaN,NaN,NaN
4,99233416,https://img3.idealista.com/blur/WEB_LISTING/0/...,20,NaN,210000.0,duplex,sale,80.0,False,2,...,"Falperra-Santa Lucía, A Coruña",Dúplex,10876,NaN,NaN,Falperra-Santa Lucía,duplex,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,27531680,https://img3.idealista.com/blur/WEB_LISTING/0/...,37,9,300000.0,penthouse,sale,185.0,False,4,...,"Agra del Orzán - Ventorrillo, A Coruña",Ático en Calle Barcelona,1963,NaN,NaN,NaN,penthouse,NaN,NaN,NaN
46,100427002,https://img3.idealista.com/blur/WEB_LISTING/0/...,22,12,205000.0,flat,sale,92.0,True,2,...,"Someso - Matogrande, A Coruña",Piso en Calle Aurelio Aguirre Galarraga s/n,VI-12-E,NaN,NaN,NaN,NaN,NaN,NaN,False
47,97963881,https://img3.idealista.com/blur/WEB_LISTING/0/...,8,1,247200.0,flat,sale,83.0,True,2,...,"Riazor - Los Rosales, A Coruña",Piso en Calle Manuel Murguía,9311305-AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,95038714,https://img3.idealista.com/blur/WEB_LISTING/0/...,29,1,210000.0,flat,sale,96.0,True,2,...,"Eirís, A Coruña","Piso en Calle Dabán, 1",Portal 5. 1º C,True,True,NaN,NaN,NaN,NaN,True
